# Understanding Stochastic Processes and Queuing Theory

## Scenario Overview: Optimizing Service Channels in a Customer Service Center

**Decision Variables:**
- $c$: Number of service channels to operate.

**Parameters:**
- $\lambda$: Average arrival rate of customers per hour.
- $\mu$: Average service rate per customer per hour.
- $C_{\text{service}}$: Fixed cost of operating one service channel per hour.
- $C_{\text{waiting}}$: Cost per hour per customer waiting in the queue.

**Objective Function:**
Minimize the total cost, which includes the service cost and the waiting cost:
$$TC = C_{\text{service}} \cdot c + C_{\text{waiting}} \cdot L_q$$

**Constraints:**
1. The number of service channels, $c$, must be a positive integer:
   $$c \in \mathbb{Z}^+$$

2. The average number of customers in the queue, $L_q$, is given by:
   $$L_q = \frac{P_0 \lambda^c}{c! \cdot (c\mu - \lambda)^2}$$
   where $P_0$ is the probability that there are zero customers in the system, calculated as:
   $$P_0 = \left[\sum_{n=0}^{c-1} \frac{(\lambda/\mu)^n}{n!} + \frac{(\lambda/\mu)^c}{c! \cdot (1 - \lambda/(c\mu))}\right]^{-1}$$

**Note:** The model assumes that customer arrivals follow a Poisson distribution, and service times are exponentially distributed, characteristic of an M/M/c queue.


In [1]:
import gurobipy as gp
from gurobipy import GRB
import math

# Given parameters
lambda_ = 50  # Average arrival rate (customers/hour)
mu = 60       # Average service rate (customers/hour)
C_service = 100  # Cost per service channel per hour
C_waiting = 10   # Waiting cost per customer per hour
max_channels = 10  # Maximum number of service channels to consider

# Function to calculate P0
def calculate_P0(rho, c):
    sum_terms = sum((rho**k) / math.factorial(k) for k in range(c))
    sum_terms += (rho**c) / (math.factorial(c) * (1 - rho/c))
    return 1 / sum_terms

# Function to calculate Lq
def calculate_Lq(rho, c, P0):
    if rho == c:  # Avoid division by zero in case rho equals c
        return float('inf')  # Return infinity if service rate equals arrival rate
    return P0 * rho**(c+1) / (math.factorial(c) * (c - rho)**2)

# Pre-calculate P0 and Lq for different values of c
precalculated_values = {}
for c in range(1, max_channels + 1):
    rho = lambda_ / mu
    P0 = calculate_P0(rho, c)
    Lq = calculate_Lq(rho, c, P0)
    precalculated_values[c] = (P0, Lq)

# Optimization Model
model = gp.Model("QueueOptimization")

# Decision variables: binary variables for each channel count
channel_vars = model.addVars(max_channels, vtype=GRB.BINARY, name="channel")

# Objective function
total_cost = gp.quicksum((C_service * c + C_waiting * precalculated_values[c][1]) * channel_vars[c-1] for c in range(1, max_channels + 1))
model.setObjective(total_cost, GRB.MINIMIZE)

# Constraint: only one channel count can be chosen
model.addConstr(channel_vars.sum() == 1, "choose_one_channel_count")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    optimal_channels = [c for c in range(1, max_channels + 1) if channel_vars[c-1].X > 0.5][0]
    print(f"Optimal number of service channels: {optimal_channels}")
    print(f"Minimum total cost: {model.ObjVal}")
else:
    print("No optimal solution found")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x4038a102
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 141.6666667
Presolve removed 1 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 141.667 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.416666666667e+02, best bound 1.416666666667e+02, gap 0.0000%
Optimal number of service ch